# Summary

Deep Meta Reinforcement Learning is about training an agent on a distribution of similar tasks, ie, interrelated Markov Decision Processes (MDPs), so that it can perform well on average on those MDPs. 

In order to do that, [Wang et al., 2016](https://arxiv.org/pdf/1611.05763v1.pdf) proposed an architecture where a recurrent neural network of type Long-Short-Term Memory (LSTM) is trained using the experience gathered by an agent performing on an environment following another Reinforcement Learning algorithm of type Actor-Critic (A2C and A3C). They ran a total of 7 experiments to analyze the behaviour of their method on different contexts. Their main purpose was to establish whether meta-RL could: 
- Deal with the exploration/exploitation trade-off,
- Understand the abstract structure of a task.
 

### The Two-Step Task
The task we reproduced is a variation of the [two-step task](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3077926/). It is a MDP with 3 states, one first-stage state $S_1$ from which the agent can take one of two different actions ($A_1$, $A_2$), each one leading to the two second-stage states ($S_2$ and $S_3$) with correlated distributions (see the figure below). Once in a second stage-state, the agent gets a reward of $1$ with probability $(p, 1-p)$ in states $(S_2,S_3)$ respectively, with $p  \in \{0.1, 0.9\}$. This reward probability is reset randomly at each episode.

![Two-Step Task](ressources/two_step_task.png)


This task brings out the behaviour of the RL algorithm used on it, because it dissociates model-based algorithms from model-free ones. 

When an agent performs a model-based algorithm, it learns an internal model of the environment he's in (transition and reward functions) and relies on it when planning its next move. On the other hand, model-free algorithms, like Q-Learning, don't store any information of the model but map values to actions in states through local updates. 

The two step task makes it easier to distinguish between these two methods because an agent following a model-based strategy takes into account the transition probability of the actions whereas an agent following a model-free algorithm focuses on the reward only. 

### Network

The writers chose the Advantage Actor-Critic Algorithm, as its structure (multiple workers learning on their own and updating the master recurrent network periodically) already does the trick. The network takes 4 inputs:
- $t$ : current timestep, with $t_{max} = 200$.
- $x_t$ : observation at timestep t.
- $r_{t-1}$ : last reward.
- $a_{t-1}$ : last action.

and gives two outputs: 
- $V$ is the value of the current state,
- $\pi$ is the softmax policy distribution.

![Architecture of the network](ressources/architecture.png)

### Test

During their tests, they measured the stay probability of their agent when following the meta-RL procedure.  

The stay probability is the probability of repeating the same action performed in the last trial, given that the last transition was common (with the highest probability of reaching the state that gives a reward more often) or uncommon. An agent following a model-free algorithm repeats his last action as long as it brings him a reward, regardless of the type of the transition he goes through (common/uncommon), so the plot gives two bars equally high when the last trial was rewarded. On the other hand, a model-based agent repeats more often actions that entailed rewards when the transition is common.

![Model-based Vs Model-free plot](ressources/stayprob_plots.png)
 <center>Canonical pattern of behavior for model-free (left) and model-based (right) learning from [here](https://www.biorxiv.org/content/biorxiv/early/2018/04/13/295964.full.pdf) </center>


According to the results we got, which are similar to the ones showed by Wang et al, despite training the LSTM algorithm using a single-threaded A3C which is a model-free procedure, the learned LSTM showed a model-based behaviour. 



### Neuroscience

In their [other version](https://www.biorxiv.org/content/biorxiv/early/2018/04/13/295964.full.pdf) of the paper, more neuroscience oriented, they present a new modelisation of the learning process in animals, where the prefrontal cortex, along with the basal ganglia and the thalamic nuclei, are represented by a recurrent neural network whose synaptic weights are adjusted by the phasic dopamine release that runs in parallel a model-free like Reinforcement Learning, based on the stimulus-response associations.